In [ ]:
import pandas as pd
import requests
from io import BytesIO, StringIO
import re
import datetime as dt
import numpy as np
import folium
import geopandas

pd.options.display.max_columns = 1000


def last_k_thursdays(k=3):
    today = dt.date.today()
    offset = (today.weekday() - 3) % 7
    latest_thursdays = [today - dt.timedelta(days=offset)]
    for w in range(1, k):
        latest_thursdays.append(latest_thursdays[0] - dt.timedelta(days=w*7))
    latest_thursdays = [dt.datetime.strftime(date, "%Y-%m-%d")  for date in latest_thursdays]
    return latest_thursdays

def zip_code_population():
    zcp = pd.read_html("https://www.massachusetts-demographics.com/zip_codes_by_population")\
    .pop(0).drop(530).drop("Rank", axis=1)
    zcp["Zip Code"] = zcp["Zip Code"].apply(lambda x: x.split(" and "))
    zcp = zcp.explode("Zip Code").reset_index(drop=True)
    zcp["Population"] = zcp["Population"].astype(int) 
    return zcp

def zip_code_town():
    zct = pd.read_html("https://www.zipcodestogo.com/Massachusetts/")[1]
    zct.columns = zct.iloc[1, :]
    zct = zct.drop([0, 1]).drop("Zip Code Map", axis=1).rename(columns={"City": "Town"})
    return zct

def geo_json_df():
    ma_geo_json_url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ma_massachusetts_zip_codes_geo.min.json"
    ma_geo_df = geopandas.read_file(ma_geo_json_url)\
    .rename(columns={"ZCTA5CE10": "Zip Code"})[["Zip Code", "geometry"]]
    return ma_geo_df

def ma_vax_data(date):
    
    converted_date = dt.datetime.strftime(
        dt.datetime.strptime(date, "%Y-%m-%d"), 
        "%B-%-d-%Y"
    ).lower()
    
    data_url = f"https://www.mass.gov/doc/weekly-covid-19-municipality-vaccination-report-{converted_date}/download"
    
    data = BytesIO(requests.get(data_url).content)
    
    sdf = pd.read_excel(
        data, 
        engine="openpyxl", 
        sheet_name="Sex - zip code", 
        header=[1, 2]
    )
    
    sdf = sdf[sdf["ZIP", "Unnamed: 0_level_1"] != "Unspecified"]

    fill_star_val = 0

    vax_cols = [
        "Individuals with at least one dose", 
        "Fully vaccinated individuals", 
        "Partially vaccinated individuals"
    ]

    for c in vax_cols:
        for g in ["'Female'", "'Male'", "Other'"]:
            sdf.loc[sdf[c][g] == "*", (c, g)] = fill_star_val
            sdf.loc[:, (c, g)] = sdf.loc[:, (c, g)].astype(float)
        sdf.loc[:, (c, "Total")] = sdf[c]["'Female'"] + sdf[c]["'Male'"] + sdf[c]["Other'"]

    sdf.loc[:, ("ZIP", "Unnamed: 0_level_1")] = \
    sdf.loc[:, ("ZIP", "Unnamed: 0_level_1")].apply(lambda x: str(x) if len(str(x)) == 5 else f"0{x}")
    
    sdf = pd.DataFrame(
    data=sdf[[("ZIP", "Unnamed: 0_level_1"), ("Individuals with at least one dose", "Total"), 
     ("Fully vaccinated individuals", "Total"), ("Partially vaccinated individuals", "Total")]].values, 
    columns=["Zip Code"] + vax_cols
    )
    
    zcp = zip_code_population()

    sdf = zcp.merge(sdf, on="Zip Code", how="inner")
    for c in vax_cols:
        sdf[f"Percent {c}"] = (sdf[c] / sdf["Population"] * 100).astype(float).round(2)
        sdf[f"Percent {c}"] = sdf[f"Percent {c}"]\
        .apply(lambda x: 100 if x > 100 else x)\
        .apply(lambda x: np.nan if x == 0 else x)
        
    sdf["date"] = date
        
    return sdf

In [ ]:
pct_cols = [
    'Percent Individuals with at least one dose',
    'Percent Fully vaccinated individuals',
    'Percent Partially vaccinated individuals'
]

dts_to_fetch = last_k_thursdays(k=3)

dfs = [
    ma_vax_data(date=w) 
    if i == 0 else ma_vax_data(date=w)[['Zip Code'] + pct_cols] 
    for i, w in enumerate(dts_to_fetch)
]

data = dfs.pop(0)

for i, prev_week in enumerate(dfs):
    data = data.merge(prev_week, on=["Zip Code"], how="left", suffixes=["", f" prev_wk_{i+1}"])
    for c in pct_cols:
        if i == 0:
            data[f"{c} prev_wk_{i+1}_delta"] = (data[c] - data[f"{c} prev_wk_{i+1}"]).astype(float).round(1)
        else:
            data[f"{c} prev_wk_{i+1}_delta"] = (data[f"{c} prev_wk_{i}"] - data[f"{c} prev_wk_{i+1}"]).astype(float).round(1)


In [ ]:
data["Delta of delta_wk2-wk1"] = (data["Percent Individuals with at least one dose prev_wk_1_delta"] - \
data["Percent Individuals with at least one dose prev_wk_2_delta"]).astype(float).round(1)

In [ ]:
zct = zip_code_town()
ma_geo_df = geo_json_df()

master_df = ma_geo_df\
.merge(data, on=["Zip Code"], how="left")\
.merge(zct, on=["Zip Code"], how="left")
master_df.head(1)

In [ ]:
ma_map = folium.Map(
    location=[42.2068067, -71.8333296], 
    zoom_start=8,
    min_zoom=8,
    min_lat=41,
    max_lat=43.5,
    min_lon=-73,
    max_lon=-70,
    max_bounds=True,
)

folium.Choropleth(
    geo_data=master_df,
    data=master_df,
#     bins=range(0, 110, 10),
    columns=['Zip Code', 'Percent Individuals with at least one dose'],
    key_on='feature.properties.Zip Code',
    fill_color='RdYlGn', fill_opacity=0.7, line_opacity=0.3,
    legend_name=f'Percent population with at least one dose (as of {dts_to_fetch[0]})'
).add_to(ma_map)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    master_df,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=[
            'Town', 
            'Percent Individuals with at least one dose', 
            'Percent Fully vaccinated individuals', 
            'Zip Code',
            'Percent Individuals with at least one dose prev_wk_1_delta',
            'Delta of delta_wk2-wk1'
        ],
        aliases=[
            'Town: ', 
            '% with at least one dose: ', 
            '% fully vaccinated: ', 
            'Zip Code: ',
            f'% with at least one dose increase (from {dts_to_fetch[1]}): ',
            f'Vaccine acceleration: '
        ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
ma_map.add_child(NIL)
ma_map.keep_in_front(NIL)
folium.LayerControl().add_to(ma_map)
ma_map

In [ ]:
ma_map.save(f"../plots/{dts_to_fetch[0].replace('-', '')}_ma_zip_vax.html")

In [ ]:
print(
    "Percent with at least 1 dose:\t"
    f"{master_df['Individuals with at least one dose'].sum()/master_df['Population'].sum():.01%}"
    "\nPercent fully vaccinated:\t"
    f"{master_df['Fully vaccinated individuals'].sum()/master_df['Population'].sum():.01%}"
)